+ ```from quantopian.pipeline.experimental import QTradableStocksUS``` の銘柄を取得
+ 前日のクローズから当日の10：30までのデータのみ取得
+ 前日クローズから0940までに５％GUしたものがFDするか確認


In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing

import pandas as pd
import numpy as np 

def make_pipeline():
    pipe = Pipeline()
    base_universe = QTradableStocksUS()
    pipe.add(USEquityPricing.close.latest, 'close')
    pipe.add(USEquityPricing.open.latest, 'open')
    
    pipe.set_screen(base_universe)
    return pipe

    

In [ ]:
pipeline_results = run_pipeline(make_pipeline(), 
                                start_date='2014-1-1', end_date='2017-12-31')


In [ ]:
## 期間中，すべての日においてQTradableStocksUSに入っていた銘柄を取り出す
dates =  pipeline_results.index.get_level_values(0).unique()
tickers = pipeline_results.ix[dates[0]].index

for date in dates[1:]:
    tickers = tickers.intersection( pipeline_results.ix[date].index)
    


In [ ]:
tickers

In [ ]:
d = dict()
for date in dates:
    d[date] = pipeline_results.ix[date].loc[tickers]
    

In [ ]:
pan = pd.Panel(d)
pan.major_axis = map(lambda x: x.symbol, pan.major_axis)


In [ ]:
pan = pan.transpose(2,0,1)


In [ ]:
pan["open"] / pan["close"].shift(1) - 1 

In [ ]:
((pan["open"] / pan["close"].shift(1) - 1 )> 0.05).sum().sort_values()

In [ ]:
l = list()
for i, row in ((pan["open"] / pan["close"].shift(1) - 1 )> 0.05).iterrows():
     l.append([i, len(row.where(row==True).dropna().index)])


In [ ]:
df = pd.DataFrame(l).set_index(0)

In [ ]:
spy = get_pricing("spy", start_date=df.index[0],end_date=df.index[-1],
                  fields="price",)
df["spy"] = spy 


In [ ]:
df.plot(secondary_y="spy")